In [ ]:
# 🧹 memory cleanup and garbage collection
import gc
import os
import sys
from dotenv import load_dotenv

def cleanup_memory():
    """memory cleanup and garbage collection"""
    # if existing C1 retrieval system, clean up
    if 'c1_retrieval' in globals():
        try:
            c1_retrieval.close()
            del globals()['c1_retrieval']
            print("✅  C1 retrieval system connection released")
        except:
            pass
    
    # if existing C2 retrieval system, clean up
    if 'c2_retrieval' in globals():
        try:
            c2_retrieval.close()
            del globals()['c2_retrieval']
            print("✅ C2 retrieval system connection released")
        except:
            pass

    # if existing C3 retrieval system, clean up
    if 'c3_retrieval' in globals():
        try:
            c3_retrieval.close()
            del globals()['c3_retrieval']
            print("✅ C3 retrieval system connection released")
        except:
            pass
    
    # if existing C4 retrieval system, clean up
    if 'c4_retrieval' in globals():
        try:
            c4_retrieval.close()
            del globals()['c4_retrieval']
            print("✅ C4 retrieval system connection released")
        except:
            pass
    
    # cache cleanup (config module, etc.)
    modules_to_reload = ['config', 'c1_retrieval', 'c2_retrieval', 
                        'prompts.c1_prompt', 'prompts.c2_prompt']
    
    for module_name in modules_to_reload:
        if module_name in sys.modules:
            try:
                del sys.modules[module_name]
                print(f"🔄 {module_name} module cache cleaned")
            except:
                pass
    
    # perform garbage collection
    collected = gc.collect()
    print(f"🧹 garbage collection completed: {collected} objects cleaned")
    
    # check cache information
    try:
        cache_file = "embedding_cache.json"
        if os.path.exists(cache_file):
            size_mb = os.path.getsize(cache_file) / 1024 / 1024
            print(f"📦 embedding cache size: {size_mb:.2f} MB")
    except:
        pass
    
    print("✨ memory cleanup completed!")

load_dotenv()
# check required environment variables
required_vars = ['OPENAI_API_KEY', 'NEO4J_URI', 'NEO4J_USER', 'NEO4J_PASSWORD']
missing_vars = [var for var in required_vars if not os.getenv(var)]

cleanup_memory()

🧹 가비지 컬렉션 완료: 0개 객체 정리
📦 임베딩 캐시 크기: 46.81 MB
✨ 메모리 정리 완료!


In [ ]:
# 🤖 LangGraph Agent RAG 시스템 통합 테스트 - 비교 버전
from pipeline import run_pipeline
from IPython.display import display, HTML, Markdown
import json

def display_comparison_results(result: dict, query: str, image_path: str = None):
    """display pipeline execution results in comparison"""
    
    # if image exists, display
    if image_path:
        print(f"🖼️ input image: {image_path}")
        # if image description is included, display
        input_text_with_image = result.get('input_text_with_image', '')
        if input_text_with_image and input_text_with_image != query:
            print(f"📸 pipeline-generated image description + query:")
            print(f"   → {input_text_with_image}")
            print()
    
    # display basic information
    task_type = result.get('task_type', 'Unknown')
    initial_response = result.get('initial_response', 'initial answer not found.')
    final_response = result.get('final_response', 'final answer not found.')
    initial_results = result.get('initial_search_results', [])
    final_results = result.get('final_search_results', [])
    debug_info = result.get('debug_info', {})
    
    # get best score round information
    final_best_round = result.get('final_best_round', 3)
    final_best_score = result.get('final_best_score', 0)
    final_best_top_k = result.get('final_best_top_k', len(final_results))
    
    print(f"🚀 LangGraph Agent RAG system comparison results")
    print("=" * 80)
    print(f"📝 user question: {query}")
    print(f"🎯 classified task: {task_type}")
    
    # display execution statistics
    if debug_info:
        classification_info = debug_info.get('classification', {})
        reflection_history = debug_info.get('reflection_history', [])
        generation_info = debug_info.get('generation', {})
        
        print(f"\n📊 실행 통계:")
        if classification_info:
            print(f"   • 태스크 분류 신뢰도: {classification_info.get('confidence', 0):.1f}%")
            print(f"   • 분류 이유: {classification_info.get('reason', 'N/A')}")
        
        if reflection_history:
            print(f"   • Reflection 반복 횟수: {len(reflection_history)}회")
            for i, reflection in enumerate(reflection_history, 1):
                print(f"     - 라운드 {i}: {reflection.get('score', 0):.1f}점 (Top-K: {reflection.get('top_k', 0)})")
        
        if generation_info:
            print(f"   • 초기 칵테일: {generation_info.get('initial_cocktails_count', 0)}개")
            print(f"   • 최종 칵테일: {generation_info.get('final_cocktails_count', 0)}개 (Round {final_best_round})")
            print(f"   • 최고 점수: {final_best_score:.1f}점 (Round {final_best_round}, Top-K: {final_best_top_k})")
    
    print("=" * 80)
    
    # 1. Round 1 답변 HTML 블록 (노란색)
    display(HTML(f"""
    <div style="background-color: #fff3cd; padding: 25px; border-radius: 12px; margin: 20px 0; border-left: 6px solid #ffc107;">
        <h2 style="color: #856404; margin-top: 0; margin-bottom: 15px;">🥉 Round 1 답변 ({len(initial_results)}개 칵테일)</h2>
        <div style="background-color: #fefefe; padding: 15px; border-radius: 8px; margin-top: 15px;">
            <div style="white-space: pre-wrap; font-family: 'Segoe UI', Arial, sans-serif; line-height: 1.7; color: #212529; ">{initial_response}</div>
        </div>
    </div>
    """))
    
    # 2. 구분선 및 개선 표시
    improvement_text = ""
    if len(final_results) > len(initial_results):
        improvement_text = f"<span style='color: #28a745; font-weight: bold;'>+{len(final_results) - len(initial_results)}개 더 많은 칵테일</span>"
    elif len(final_results) == len(initial_results):
        improvement_text = f"<span style='color: #6c757d;'>동일한 개수, 최고 품질 선택 (Round {final_best_round})</span>"
    else:
        improvement_text = f"<span style='color: #dc3545; font-weight: bold;'>{len(final_results) - len(initial_results):+d}개 칵테일</span>"
    
    display(HTML(f"""
    <div style="text-align: center; margin: 30px 0;">
        <hr style="border: 2px solid #007bff; margin-bottom: 15px;">
        <h2 style="color: #007bff; margin: 10px 0;">⚡ REFLECTION 개선 효과 ⚡</h2>
        <p style="color: #6c757d; font-size: 16px;">{improvement_text}</p>
        <hr style="border: 2px solid #007bff; margin-top: 15px;">
    </div>
    """))
    
    # 3. 최종 답변 HTML 블록 (파란색) - 최고 점수 라운드 정보 표시
    display(HTML(f"""
    <div style="background-color: #d1ecf1; padding: 25px; border-radius: 12px; margin: 20px 0; border-left: 6px solid #17a2b8;">
        <h2 style="color: #0c5460; margin-top: 0; margin-bottom: 15px;">🏆 최종 답변 (Round {final_best_round} - {len(final_results)}개 칵테일, {final_best_score:.1f}점)</h2>
        <div style="background-color: #fefefe; padding: 15px; border-radius: 8px; margin-top: 15px;">
            <div style="white-space: pre-wrap; font-family: 'Segoe UI', Arial, sans-serif; line-height: 1.7; color: #212529; ">{final_response}</div>
        </div>
    </div>
    """))
    
    # 4. 개선 요약 표시 - 최고 점수 기준으로 계산
    if debug_info and reflection_history:
        first_score = reflection_history[0].get('score', 0) if reflection_history else 0
        best_score = final_best_score  # 최고 점수 사용
        score_change = best_score - first_score
        
        change_color = "#28a745" if score_change >= 0 else "#dc3545"
        change_icon = "📈" if score_change >= 0 else "📉"
        
        # 점수 변화가 없는 경우 (최고 점수가 초기 점수와 같음)
        if score_change == 0:
            change_icon = "🎯"
            change_color = "#17a2b8"
        
        display(HTML(f"""
        <div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; margin: 20px 0; border: 1px solid #dee2e6;">
            <h3 style="color: #495057; margin-top: 0;">📊 개선 요약</h3>
            <div style="display: flex; justify-content: space-between; align-items: center;">
                <div>
                    <strong>칵테일 개수:</strong> {len(initial_results)}개 → {len(final_results)}개 
                    <span style="color: {change_color};">({len(final_results) - len(initial_results):+d}개)</span>
                </div>
                <div>
                    <strong>품질 점수:</strong> {first_score:.1f}점 → {best_score:.1f}점 
                    <span style="color: {change_color};">{change_icon} ({score_change:+.1f}점)</span>
                </div>
            </div>
            <div style="margin-top: 10px; font-size: 14px; color: #6c757d;">
                💡 <strong>선택된 라운드:</strong> Round {final_best_round} (Top-K: {final_best_top_k}) - 최고 품질 결과
            </div>
        </div>
        """))
    
    # 5. 디버그 정보 표시 (접을 수 있는 형태)
    if debug_info:
        debug_json = json.dumps(debug_info, indent=2, ensure_ascii=False)
        display(HTML(f"""
        <details style="margin: 20px 0;">
            <summary style="cursor: pointer; font-weight: bold; color: #666; padding: 10px; background-color: #f8f9fa; border-radius: 5px;">
                🔍 상세 디버그 정보 (클릭해서 보기)
            </summary>
            <pre style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; overflow-x: auto; margin-top: 10px; ">{debug_json}</pre>
        </details>
        """))

class CocktailRAG:
    """간편 칵테일 RAG 시스템 인터페이스"""
    
    def __init__(self):
        self.image = None
        self.query = ""
    
    def run(self):
        """설정된 image와 query로 RAG 시스템 실행"""
        if not self.query:
            print("⚠️ query를 설정해주세요!")
            print("사용법:")
            print("  rag.image = '/path/to/image.jpg'  # 이미지가 있는 경우")
            print("  rag.query = '질문 내용'")
            print("  rag.run()")
            return None
        
        print("🧪 LangGraph Agent RAG 시스템 실행")
        print("=" * 60)
        
        # 이미지 경로만 표시 (파이프라인에서 실제 설명 생성)
        if self.image:
            print(f"🖼️ 이미지 경로: {self.image}")
        
        # 파이프라인 실행
        result = run_pipeline(self.query, self.image)
        
        # 결과 표시
        display_comparison_results(result, self.query, self.image)
        
        return result

# 전역 인스턴스 생성
rag = CocktailRAG()

print("✅ LangGraph Agent RAG 시스템 함수가 로드되었습니다!")
print("\n💡 사용법:")
print("  rag.image = '/path/to/image.jpg'  # 이미지가 있는 경우 (없으면 None)")
print("  rag.query = '질문 내용'")
print("  rag.run()")

✅ LangGraph Agent RAG 시스템 함수가 로드되었습니다!

💡 사용법:
  rag.image = '/path/to/image.jpg'  # 이미지가 있는 경우 (없으면 None)
  rag.query = '질문 내용'
  rag.run()


In [ ]:
# c 1
rag.image = './data/image/c1-6.jpeg' 
rag.query = "Looks like a red cocktail with a lemon garnish. Find similar red-tone cocktails."  
rag.run() 

🧪 LangGraph Agent RAG 시스템 실행
🖼️ 이미지 경로: /Users/yujin/Desktop/cocktail_rec_agentrag/image/Negroni.jpeg
🚀 파이프라인 시작
📝 질문: Looks like a red cocktail with a lemon garnish. Find similar red-tone cocktails.
🖼️ 이미지: /Users/yujin/Desktop/cocktail_rec_agentrag/image/Negroni.jpeg
🖼️ 이미지 설명 생성 중: /Users/yujin/Desktop/cocktail_rec_agentrag/image/Negroni.jpeg
✅ 이미지 설명 생성 완료
🤖 태스크 분류 중...
📝 쿼리: The image depicts a cocktail served in a clear glass with a sturdy base, featuring a vibrant reddish-brown liquid. Several large ice cubes, one of which is particularly prominent and slightly clouded, rest within the drink. A wedge of bright yellow-orange lemon is positioned on the rim of the glass, adding a contrasting color to the composition. The backdrop showcases a wooden surface, lending a warm and rustic feel, with light reflecting off the drink's surface, enhancing its inviting appearance. Looks like a red cocktail with a lemon garnish. Find similar red-tone cocktails.
✅ 태스크 분류 완료: C1 (신뢰도: 90.0%)
📋 분류

✅ 초기 답변 생성 완료
🏆 최종 답변 생성 중... (Round 2, 4개 칵테일, 72.5점)
🔍 태스크 프롬프트 확인 (C1):
   - 프롬프트 길이: 1023 글자
   - context 플레이스홀더 포함: True
   - question 플레이스홀더 포함: True
   - context 위치 주변: ...earance.

Question: {question}

Search Results:  
{context}

Answer:
...
🔍 칵테일 컨텍스트 확인:
   - 칵테일 수: 4
   - 칵테일 이름들: ['diesel', 'vermouth cassis', 'boston sour', 'campari beer']
   - 포맷된 컨텍스트 길이: 2634 글자
   - 컨텍스트 미리보기 (처음 200자): 1. **diesel**
   - category: beer
   - glass_type: pint glass
   - alcoholic: Alcoholic
   - ingredients: lager, blackcurrant cordial, cider
   - instructions: Pour the lager first then add the blackc...
🎯 최종 응답 생성 중... (C1)


✅ 최종 답변 생성 완료
✅ 파이프라인 완료
🖼️ 입력 이미지: /Users/yujin/Desktop/cocktail_rec_agentrag/image/Negroni.jpeg
📸 파이프라인에서 생성된 이미지 설명 + 쿼리:
   → The image depicts a cocktail served in a clear glass with a sturdy base, featuring a vibrant reddish-brown liquid. Several large ice cubes, one of which is particularly prominent and slightly clouded, rest within the drink. A wedge of bright yellow-orange lemon is positioned on the rim of the glass, adding a contrasting color to the composition. The backdrop showcases a wooden surface, lending a warm and rustic feel, with light reflecting off the drink's surface, enhancing its inviting appearance. Looks like a red cocktail with a lemon garnish. Find similar red-tone cocktails.

🚀 LangGraph Agent RAG 시스템 비교 결과
📝 사용자 질문: Looks like a red cocktail with a lemon garnish. Find similar red-tone cocktails.
🎯 분류된 태스크: C1

📊 실행 통계:
   • 태스크 분류 신뢰도: 90.0%
   • 분류 이유: The question focuses on finding visually similar cocktails based on the color red and the visual appeal

{'user_query': {'text': 'Looks like a red cocktail with a lemon garnish. Find similar red-tone cocktails.',
  'image': '/Users/yujin/Desktop/cocktail_rec_agentrag/image/Negroni.jpeg'},
 'input_text': 'Looks like a red cocktail with a lemon garnish. Find similar red-tone cocktails.',
 'input_image': '/Users/yujin/Desktop/cocktail_rec_agentrag/image/Negroni.jpeg',
 'task_type': 'C1',
 'task_confidence': 90.0,
 'task_reason': 'The question focuses on finding visually similar cocktails based on the color red and the visual appeal of the drink, making C1 the appropriate classification.',
 'input_text_with_image': "The image depicts a cocktail served in a clear glass with a sturdy base, featuring a vibrant reddish-brown liquid. Several large ice cubes, one of which is particularly prominent and slightly clouded, rest within the drink. A wedge of bright yellow-orange lemon is positioned on the rim of the glass, adding a contrasting color to the composition. The backdrop showcases a wooden sur

In [ ]:
# c 2
rag.image = './image/mojito.jpeg' 
rag.query = "Please find highball glass cocktails that include rum and lime like the photo."  
rag.run()  

🧪 LangGraph Agent RAG 시스템 실행
🖼️ 이미지 경로: /Users/yujin/Desktop/cocktail_rec_agentrag/image/mojito.jpeg
🚀 파이프라인 시작
📝 질문: Please find highball glass cocktails that include rum and lime like the photo.
🖼️ 이미지: /Users/yujin/Desktop/cocktail_rec_agentrag/image/mojito.jpeg
🖼️ 이미지 설명 생성 중: /Users/yujin/Desktop/cocktail_rec_agentrag/image/mojito.jpeg
✅ 이미지 설명 생성 완료
🤖 태스크 분류 중...
📝 쿼리: The image features a tall glass cocktail filled with a refreshing drink. The drink is transparent with a light green hue, likely indicating the presence of fresh mint and lime. Inside the glass, several thin slices of cucumber and vibrant green mint leaves are visible, providing a contrasting color and texture. The background is softly blurred but suggests a warm, inviting setting with hints of light and additional greenery, enhancing the overall appeal of the cocktail. This description aligns with a traditional Mojito. Please find highball glass cocktails that include rum and lime like the photo.
✅ 태스크 분류 완료: C2 (

✅ 초기 답변 생성 완료
🏆 최종 답변 생성 중... (Round 2, 1개 칵테일, 63.8점)
🔍 태스크 프롬프트 확인 (C2):
   - 프롬프트 길이: 835 글자
   - context 플레이스홀더 포함: True
   - question 플레이스홀더 포함: True
   - context 위치 주변: ... matching.

Question: {question}

Search Results:
{context}

Answer:
...
🔍 칵테일 컨텍스트 확인:
   - 칵테일 수: 1
   - 칵테일 이름들: ['bahama mama']
   - 포맷된 컨텍스트 길이: 621 글자
   - 컨텍스트 미리보기 (처음 200자): 1. **bahama mama**
   - category: cocktail
   - glass_type: highball glass
   - alcoholic: Alcoholic
   - ingredients: sweet and sour, orange juice, rum, pineapple juice, dark rum, banana liqueur, gre...
🎯 최종 응답 생성 중... (C2)


✅ 최종 답변 생성 완료
✅ 파이프라인 완료
🖼️ 입력 이미지: /Users/yujin/Desktop/cocktail_rec_agentrag/image/mojito.jpeg
📸 파이프라인에서 생성된 이미지 설명 + 쿼리:
   → The image features a tall glass cocktail filled with a refreshing drink. The drink is transparent with a light green hue, likely indicating the presence of fresh mint and lime. Inside the glass, several thin slices of cucumber and vibrant green mint leaves are visible, providing a contrasting color and texture. The background is softly blurred but suggests a warm, inviting setting with hints of light and additional greenery, enhancing the overall appeal of the cocktail. This description aligns with a traditional Mojito. Please find highball glass cocktails that include rum and lime like the photo.

🚀 LangGraph Agent RAG 시스템 비교 결과
📝 사용자 질문: Please find highball glass cocktails that include rum and lime like the photo.
🎯 분류된 태스크: C2

📊 실행 통계:
   • 태스크 분류 신뢰도: 90.0%
   • 분류 이유: The question explicitly mentions a specific glass type (highball glass) and requests 

{'user_query': {'text': 'Please find highball glass cocktails that include rum and lime like the photo.',
  'image': '/Users/yujin/Desktop/cocktail_rec_agentrag/image/mojito.jpeg'},
 'input_text': 'Please find highball glass cocktails that include rum and lime like the photo.',
 'input_image': '/Users/yujin/Desktop/cocktail_rec_agentrag/image/mojito.jpeg',
 'task_type': 'C2',
 'task_confidence': 90.0,
 'task_reason': 'The question explicitly mentions a specific glass type (highball glass) and requests cocktails that include rum and lime, which aligns with the glass type and ingredient combination.',
 'input_text_with_image': 'The image features a tall glass cocktail filled with a refreshing drink. The drink is transparent with a light green hue, likely indicating the presence of fresh mint and lime. Inside the glass, several thin slices of cucumber and vibrant green mint leaves are visible, providing a contrasting color and texture. The background is softly blurred but suggests a warm,

In [5]:
# c 3
rag.image = '' 
rag.query = "Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails."  
rag.run()  

🧪 LangGraph Agent RAG 시스템 실행
🚀 파이프라인 시작
📝 질문: Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails.
🤖 태스크 분류 중...
📝 쿼리: Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails.
✅ 태스크 분류 완료: C3 (신뢰도: 90.0%)
📋 분류 이유: The question provides a list of ingredients (gin and lemon) and seeks to explore related cocktails based on shared ingredients in the sour family, fitting the multi-hop ingredient expansion search.

🔍 검색 시작: C3 (Top-3)
🔄 C3 검색 시스템 초기화 중...
✅ C3 검색 시스템 초기화 완료
🔧 Top-K 업데이트: 3
C3 Retrieval (Multi-hop 재료 확장): 사용자 질문 - Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails.
1단계 - 키워드 추출: {'ingredients': ['gin', 'lemon'], 'cocktail_names': []}
   → 재료 기반 초기 검색: 6개 칵테일
3-1단계 - 초기 재료 검색: 6개
   → Multi-hop 확장 결과: 8개 칵테일
      상위 5개: ['sweet sangria', 'jitterbug', 'frozen daiquiri', 'acapulco', 'rose']
3-2단계 - Multi-hop 확장: 8개
4단계 - 후보 정리:

✅ 초기 답변 생성 완료
🏆 최종 답변 생성 중... (Round 2, 4개 칵테일, 78.8점)
🔍 태스크 프롬프트 확인 (C3):
   - 프롬프트 길이: 1307 글자
   - context 플레이스홀더 포함: True
   - question 플레이스홀더 포함: True
   - context 위치 주변: ...expansion.

Question: {question}

Search Results:
{context}

Answer:
...
🔍 칵테일 컨텍스트 확인:
   - 칵테일 수: 4
   - 칵테일 이름들: ['gin fizz', 'gin sling', 'jitterbug', 'lady love fizz']
   - 포맷된 컨텍스트 길이: 2504 글자
   - 컨텍스트 미리보기 (처음 200자): 1. **gin fizz**
   - category: ordinary drink
   - glass_type: highball glass
   - alcoholic: Alcoholic
   - instructions: Shake all ingredients with ice cubes, except soda water. Pour into glass. Top...
🎯 최종 응답 생성 중... (C3)


✅ 최종 답변 생성 완료
✅ 파이프라인 완료
🚀 LangGraph Agent RAG 시스템 비교 결과
📝 사용자 질문: Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails.
🎯 분류된 태스크: C3

📊 실행 통계:
   • 태스크 분류 신뢰도: 90.0%
   • 분류 이유: The question provides a list of ingredients (gin and lemon) and seeks to explore related cocktails based on shared ingredients in the sour family, fitting the multi-hop ingredient expansion search.
   • Reflection 반복 횟수: 3회
     - 라운드 1: 78.8점 (Top-K: 3)
     - 라운드 2: 78.8점 (Top-K: 4)
     - 라운드 3: 77.5점 (Top-K: 5)
   • 초기 칵테일: 3개
   • 최종 칵테일: 4개 (Round 2)
   • 최고 점수: 78.8점 (Round 2, Top-K: 4)


{'user_query': {'text': 'Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails.',
  'image': ''},
 'input_text': 'Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails.',
 'input_image': '',
 'task_type': 'C3',
 'task_confidence': 90.0,
 'task_reason': 'The question provides a list of ingredients (gin and lemon) and seeks to explore related cocktails based on shared ingredients in the sour family, fitting the multi-hop ingredient expansion search.',
 'input_text_with_image': 'Seed with gin and lemon, extract shared ingredients in the sour family, and expand to related cocktails.',
 'current_top_k': 5,
 'search_results': [{'name': 'gin fizz',
   'description': 'The Gin Fizz is a classic member of the Fizz family, characterized by its bubbly, refreshing nature. Originating in the 19th century, this gin-based cocktail is believed to have emerged from the popularity of fizzy drinks like lemo

In [ ]:
# c 4
rag.image = './image/manhattan.jpeg' 
rag.query = "This looks similar to Manhattan; please suggest alternatives with a similar number of ingredients."  
rag.run() 

🧪 LangGraph Agent RAG 시스템 실행
🖼️ 이미지 경로: /Users/yujin/Desktop/cocktail_rec_agentrag/image/manhattan.jpeg
🚀 파이프라인 시작
📝 질문: This looks similar to Manhattan; please suggest alternatives with a similar number of ingredients.
🖼️ 이미지: /Users/yujin/Desktop/cocktail_rec_agentrag/image/manhattan.jpeg
🖼️ 이미지 설명 생성 중: /Users/yujin/Desktop/cocktail_rec_agentrag/image/manhattan.jpeg
✅ 이미지 설명 생성 완료
🤖 태스크 분류 중...
📝 쿼리: The image presents a cocktail served in a clear, short glass filled with ice cubes. The drink exhibits a warm, amber hue, suggesting a mixture of spirits, potentially accompanied by citrus flavors. A slice of orange rests on the rim, adding a vibrant pop of bright orange color, while a gold cocktail stirrer is positioned diagonally across the glass. The overall composition is clean and minimalist, with a smooth white background that emphasizes the cocktail's rich color and textures. This looks similar to Manhattan; please suggest alternatives with a similar number of ingredients.
✅ 태스크 

✅ 초기 답변 생성 완료
🏆 최종 답변 생성 중... (Round 2, 4개 칵테일, 81.2점)
🔍 태스크 프롬프트 확인 (C4):
   - 프롬프트 길이: 943 글자
   - context 플레이스홀더 포함: True
   - question 플레이스홀더 포함: True
   - context 위치 주변: ...endations.

Question: {question}

Search Results:
{context}

Answer:
...
🔍 칵테일 컨텍스트 확인:
   - 칵테일 수: 4
   - 칵테일 이름들: ['manhattan', 'old fashioned', 'french negroni', 'martinez 2']
   - 포맷된 컨텍스트 길이: 2735 글자
   - 컨텍스트 미리보기 (처음 200자): 1. **manhattan**
   - category: cocktail
   - glass_type: cocktail glass
   - alcoholic: Alcoholic
   - ingredients:
     • 3/4 oz sweet vermouth
     • 2 1/2 oz blended bourbon
     • dash angostura ...
🎯 최종 응답 생성 중... (C4)


✅ 최종 답변 생성 완료
✅ 파이프라인 완료
🖼️ 입력 이미지: /Users/yujin/Desktop/cocktail_rec_agentrag/image/manhattan.jpeg
📸 파이프라인에서 생성된 이미지 설명 + 쿼리:
   → The image presents a cocktail served in a clear, short glass filled with ice cubes. The drink exhibits a warm, amber hue, suggesting a mixture of spirits, potentially accompanied by citrus flavors. A slice of orange rests on the rim, adding a vibrant pop of bright orange color, while a gold cocktail stirrer is positioned diagonally across the glass. The overall composition is clean and minimalist, with a smooth white background that emphasizes the cocktail's rich color and textures. This looks similar to Manhattan; please suggest alternatives with a similar number of ingredients.

🚀 LangGraph Agent RAG 시스템 비교 결과
📝 사용자 질문: This looks similar to Manhattan; please suggest alternatives with a similar number of ingredients.
🎯 분류된 태스크: C4

📊 실행 통계:
   • 태스크 분류 신뢰도: 90.0%
   • 분류 이유: The question explicitly mentions a target cocktail (Manhattan) and requests alte

{'user_query': {'text': 'This looks similar to Manhattan; please suggest alternatives with a similar number of ingredients.',
  'image': '/Users/yujin/Desktop/cocktail_rec_agentrag/image/manhattan.jpeg'},
 'input_text': 'This looks similar to Manhattan; please suggest alternatives with a similar number of ingredients.',
 'input_image': '/Users/yujin/Desktop/cocktail_rec_agentrag/image/manhattan.jpeg',
 'task_type': 'C4',
 'task_confidence': 90.0,
 'task_reason': 'The question explicitly mentions a target cocktail (Manhattan) and requests alternatives with a similar number of ingredients, fitting the criteria for C4.',
 'input_text_with_image': "The image presents a cocktail served in a clear, short glass filled with ice cubes. The drink exhibits a warm, amber hue, suggesting a mixture of spirits, potentially accompanied by citrus flavors. A slice of orange rests on the rim, adding a vibrant pop of bright orange color, while a gold cocktail stirrer is positioned diagonally across the gl